In [ ]:
import numpy as np
import os, glob
from skimage import io
import xtiff

In [ ]:
def write_composite(data_path, plateID, pipeline_name, well, field_num):
    c_filename = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files",plateID+"_C_"+well+"_"+str(field_num)+"_reg_stack.ome.tiff")
    print("creating composite movie file for "+c_filename)
    r_src = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files",plateID+"_R_"+well+"_"+str(field_num)+"_reg_stack.tif")
    g_src = r_src.replace("_R_","_G_")
    p_src = r_src.replace("_R_","_P_")

    mask_src = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files/tracking",well,"field_"+str(field_num),"filtered_masks")
    mask_filenames = sorted(glob.glob(mask_src+"/mask*"))
    r_stack = io.imread(r_src)
    g_stack = io.imread(g_src)
    p_stack = io.imread(p_src)
    
    mask_list = []
    for fn in mask_filenames:
        #read in the mask image
        masks = io.imread(fn)
        mask_list.append(masks)
    mask_stack = np.stack(mask_list)
    composite_stack = np.stack((r_stack, g_stack, mask_stack, p_stack), axis = 1)
    
    res = xtiff.to_tiff(composite_stack, c_filename)
    return(res)

In [ ]:
data_path = "/home/exacloud/gscratch/HeiserLab/images/"
plateID_list = ("HC00701", "HC00801", "HC00901", "HC01001", "HC01301", "HC01401")
#plateID_list = (["HC00701"])
pipeline_name = "CKn"
well_list = ("A1", "D2", "D4", "D6")
#well_list = (["D2"])

field_num = 1

res = list(map(write_composite, [data_path]*len(well_list)*len(plateID_list), [element for element in plateID_list for i in range(len(well_list))], [pipeline_name]*len(well_list)*len(plateID_list), well_list*len(plateID_list), [field_num]*len(well_list)*len(plateID_list)))